In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('mini.csv')

In [3]:
data.head()

,customer_id,occurence,cost,item_id
0,416705,2017-05-07 21:58:10.000000,299.0,515274
1,13891,2018-02-10 17:35:11.000000,1090.0,828115
2,9081,2017-12-21 17:13:44.000000,499.0,695501
3,470904,2017-10-31 10:39:49.000000,290.0,899821
4,58500,2018-03-09 20:57:29.000000,150.0,518554


In [4]:
data.info()
hz=pd.DataFrame(data.groupby('customer_id')['cost'].mean())
hz['number_of_transactions']=data.groupby('customer_id')['cost'].count()
hz['total']=data.groupby('customer_id')['cost'].sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990712 entries, 0 to 1990711
Data columns (total 4 columns):
customer_id    int64
occurence      object
cost           float64
item_id        object
dtypes: float64(1), int64(1), object(2)
memory usage: 45.6+ MB


In [5]:
data['occurence']=pd.to_datetime(data.occurence,yearfirst=True)

In [6]:
data['year']=[d.year for d in data.occurence]
data['month']=[d.month for d in data.occurence]

In [7]:
data.drop('occurence',axis=1,inplace=True)


In [8]:
data.head()

,customer_id,cost,item_id,year,month
0,416705,299.0,515274,2017,5
1,13891,1090.0,828115,2018,2
2,9081,499.0,695501,2017,12
3,470904,290.0,899821,2017,10
4,58500,150.0,518554,2018,3


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990712 entries, 0 to 1990711
Data columns (total 5 columns):
customer_id    int64
cost           float64
item_id        object
year           int64
month          int64
dtypes: float64(1), int64(3), object(1)
memory usage: 68.3+ MB


In [10]:
data=data[data['year']==2018]
data=data[data['month']<=6]

In [11]:
five_month=data[data['month']<=5]
target_month=data[data['month']==6]

In [12]:
target_month=target_month[['customer_id']].drop_duplicates(subset=['customer_id'], keep=False)
five_month=five_month[['customer_id']].drop_duplicates(subset=['customer_id'],keep=False)
target_month['target']='yes'
result=pd.merge(five_month,target_month, how='left', on=['customer_id'])
result['target']=(result.target.isnull()).astype('int')

In [13]:
total=pd.merge(result,hz,how='inner',on=['customer_id'])

In [14]:
total.head()
total.drop('customer_id',axis=1,inplace=True)

In [36]:
total.head()

,target,cost,number_of_transactions,total
0,1,590.000000,1,590.0
1,1,500.000000,1,500.0
2,1,3858.333333,3,11575.0
3,1,1346.666667,3,4040.0
4,1,3890.000000,1,3890.0


In [37]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46059 entries, 0 to 46058
Data columns (total 4 columns):
target                    46059 non-null int32
cost                      46059 non-null float64
number_of_transactions    46059 non-null int64
total                     46059 non-null float64
dtypes: float64(2), int32(1), int64(1)
memory usage: 1.6 MB


In [15]:
y=total['target'].astype('int')

In [16]:
X=total.drop('target',axis=1)

In [17]:
X.shape,y.shape

((46059, 3), (46059,))

In [18]:
from sklearn.model_selection import train_test_split, cross_val_score

In [19]:
X_train,X_valid,y_train,y_valid=train_test_split(X, y, 
                                                  test_size=0.3,
                                                  random_state=17)

In [20]:
X_train.shape,y_train.shape

((32241, 3), (32241,))

In [21]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
first_tree=DecisionTreeClassifier(random_state=17)

In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
tree_params={'max_depth':np.arange(1,11),'max_features':([.5,.7,1])}

In [25]:
tree_grid=GridSearchCV(first_tree,tree_params,cv=5,n_jobs=-1)

In [26]:
%%time
tree_grid.fit(X_train,y_train)

Wall time: 10 s


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), 'max_features': [0.5, 0.7, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
tree_grid.best_score_,tree_grid.best_params_

(0.9317018702893831, {'max_depth': 1, 'max_features': 0.5})

In [28]:
tree_valid_pred=tree_grid.predict(X_valid)

In [29]:
from sklearn.metrics import accuracy_score

In [30]:
accuracy_score(y_valid,tree_valid_pred)

0.9256766536401795

In [31]:
from sklearn.tree import export_graphviz

In [34]:
export_graphviz(tree_grid.best_estimator_,out_file='tree.dot',
               feature_names=X.columns, filled=True)

In [35]:
!dot -Tpng tree.dot -o tree.png

<img src = 'tree.png'>

In [137]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46059 entries, 0 to 46058
Data columns (total 4 columns):
target                    46059 non-null int32
cost                      46059 non-null float64
number_of_transactions    46059 non-null int64
total                     46059 non-null float64
dtypes: float64(2), int32(1), int64(1)
memory usage: 1.6 MB
